# Analysis

In the previous notebook, we created a file with your measured heartbeat. In this notebook, you will analyze that data which you collected. Here is how to load that data back into Python:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y = np.loadtxt("heartbeat.txt")

print(y[:20])  # show me just the first 20 entries

A good first step for understanding data is to visualize it -- that is, make a plot. `matplotlib` makes this easy:

In [ ]:
plt.plot(y)
plt.show()

Unfortunately the x-axis is all wrong -- it is just counting measurements one by one, not giving us the actual time of each measurement. To fix this, we can make an array which has the time each measurement was taken.

### Exercise

Make an array called `t` which has the time corresponding to each measurement. If you feel stuck, look at the previous notebook for inspiration.

In [ ]:
t = np.arange(len(y)) / 200  # ANSWER HERE

plt.plot(t, y)
plt.show()

Now zoom in to see just the first second. You can use the command `plt.xlim`.

In [ ]:
help(plt.xlim)

In [ ]:
plt.plot(t, y)
plt.xlim(0, 1)
plt.show()

There is some jaggedness to the signal. This is present because the Pulse Sensor has some noise in its circuitry -- it introduces some random error. We can get rid of it using the same trick we always use to manage random error: averaging. Specifically, we can average each point with its neighbors. The following code averages each point with its ten nearest neighbors on each side. Plot the result.

In [ ]:
from heartbeat import rolling_centered_average

averaged = rolling_centered_average(y, 3)
plt.plot(t, averaged)
plt.xlim(0, 1)
plt.show()

Notice that the signal is smoother, but there is still some unwanted jiggle to it. Increase the number of terms in the average until it goes up and down smoothly.

In [ ]:
averaged = rolling_centered_average(y, 30)  ## ANSWER HERE!

plt.plot(t, averaged)
plt.xlim(0.1, 1.1)
plt.show()

Our main objective is to identify the frequency of this signal -- that is, the heart rate. To do that, let's see if we can mark the local maxima. Here is an example of a function which will identify those points where the function is increasing (where the current value is bigger than the previous). It may give a warning about invalid values -- that's because we don't have any good average values for the values at the ends, so they are filled in with the "not a number" value, `NaN`.

In [ ]:
increasing = np.zeros(
    averaged.shape, dtype=bool
)  # this creates an array filled with False
increasing[1:] = averaged[1:] > averaged[:-1]
plt.plot(t, averaged)
plt.plot(t[increasing], averaged[increasing], "o")
plt.xlim(0.2, 0.9)
plt.show()

We can see where it is decreasing by looking at the complement. Make a plot similar to the above, showing where the function is decreasing.

In [ ]:
decreasing = ~increasing

## answer here
plt.plot(t, averaged)
plt.plot(t[decreasing], averaged[decreasing], "o")
plt.xlim(0.2, 0.9)
plt.show()

Now it is just a matter of finding where the switch occurs:

In [ ]:
maxima = np.zeros(averaged.shape, dtype=bool)
maxima[:-1] = increasing[:-1] & decreasing[1:]

plt.plot(t, averaged)
plt.plot(t[maxima], averaged[maxima], "o")
plt.show()

finally, we can remove any smaller wobbles by using a threshold -- only counting maxima above a certain value. This will get us only those top peaks.

In [ ]:
peak_indices = maxima & (averaged > averaged[maxima].mean())

plt.plot(t, averaged)
plt.plot(t[peak_indices], averaged[peak_indices], "o")
plt.show()

Now that we have identified the peaks, we can take the times these have occurred and compute the BPM.

In [ ]:
peak_times = t[peak_indices]
peak_times

## exercise

use the peak times to compute your BPM.